In [ ]:
import tensorflow as tf
import random

In [37]:
words = open('../names.txt').read().split()

In [38]:
stoi = {s: i+1 for i, s in enumerate(sorted(set(''.join(words))))}
stoi['.'] = 0
itos = {s: i for i, s in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [39]:
tf.random.set_global_generator(42)
w = tf.Variable(tf.random.normal((3, 2)), name='w')
a = tf.Variable(tf.random.normal((2, 3)), name='a')
a@w

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-1.1136764 , -0.30688876],
       [ 0.5948322 , -0.1667712 ]], dtype=float32)>

In [40]:

def build_dataset(words: list[str]):
    X, Y = [], []
    context_size = 3
    for w in words:
        context = [0] * context_size
        for ch in w + ".":
            X.append(context)
            Y.append(stoi[ch])
            # output = [itos[i] for i in context]
            # print(f"{''.join(output)} ----> {ch}")
            context = context[1:] + [stoi[ch]]
    return tf.Variable(X), tf.Variable(Y)


n1, n2 = int(0.8 * len(words)), int(0.9 * len(words))
random.shuffle(words)

X_train, Y_train = build_dataset(words=words[:n1])
X_dev, Y_dev = build_dataset(words=words[n1:n2])
X_test, Y_test = build_dataset(words=words[n2:])

# X_train, Y_train = X_train.to(device=device), Y_train.to(device=device)
# X_dev, Y_dev = X_dev.to(device=device), Y_dev.to(device=device)
# X_test, Y_test = X_test.to(device=device), Y_test.to(device=device)

X_train.shape, X_dev.shape, X_test.shape

(TensorShape([182339, 3]), TensorShape([22858, 3]), TensorShape([22949, 3]))

In [77]:
C = tf.Variable(tf.random.uniform(shape=(27, 5)), trainable=True)
W1 = tf.Variable(tf.random.uniform(shape=(15, 200)), trainable=True)
b1 = tf.Variable(tf.random.uniform(shape=(200,)), trainable=True)
W2 = tf.Variable(tf.random.uniform(shape=(200, 27)), trainable=True)
b2 = tf.Variable(tf.random.uniform(shape=(27,)), trainable=True)
parameters = [C, W1, b1, W2, b2]

In [84]:
ll = tf.gather(C, X_train)
tf.reshape(ll[0], shape=(-1, 15))

<tf.Tensor: shape=(1, 15), dtype=float32, numpy=
array([[0.35408378, 0.20130026, 0.5752524 , 0.20098436, 0.72094214,
        0.35408378, 0.20130026, 0.5752524 , 0.20098436, 0.72094214,
        0.35408378, 0.20130026, 0.5752524 , 0.20098436, 0.72094214]],
      dtype=float32)>

In [78]:
sum(tf.size(p) for p in parameters)

<tf.Tensor: shape=(), dtype=int32, numpy=8762>

In [ ]:

def forward(inputs):
    emb = tf.gather(C, inputs)
    h = tf.tanh(tf.reshape(emb, shape=(-1, 3*5) @ W1+b1))
    logits = h@W2+b2
    return logits

In [ ]:
stepi, lossi, lossd = [], [], []
ix = tf.random.uniform(shape=(32,), minval=0, maxval=5, dtype=tf.int32)
print(tf.gather(C, ix).shape)

(32, 5)


In [ ]:
cross_entropy = tf.keras.losses.CategoricalCrossentropy

n_iter = 100

for i in range(n_iter):
    # minibatch
    ix = tf.random.uniform(shape=(32,), minval=0, maxval=5, dtype=tf.int32)

    inputs = tf.gather(X_train, ix)
    targets = tf.gatehr(Y_train, ix)
    logits = forward(inputs)
    loss_train = cross_entropy(targets, logits)

    logits_dev = forward(X_dev)
    loss_dev = cross_entropy(X_dev, logits_dev)